## Assignment 2 - Final Iter Visual Dashboard
![Nightingale](images/nightingale.png)
For the final iteration, I chose to use Dash to create the visual dashboard, as it integrates effectively with Plotly. I received assistance from AI to develop the code, since I had not previously worked with Dash. After several revisions and requests to refine the output, the final product achieved the result I was aiming for. I also modified the colour scheme to make it brighter, as I found both the previous version and the original visual somewhat dull.

This dashboard expands upon the original by incorporating a stacked bar chart, which presents the same data in a more straightforward manner and allows for easier comparison of exact values. However, I still believe that Nightingale’s original coxcomb design remains the most effective way to display this data. It is visually striking and conveys seasonal differences more clearly than the stacked bar chart.

The dashboard, which opens in a new tab, includes a brief animation upon loading and initially displays Nightingale’s coxcomb. It features dropdown menus that allow users to interact with different death types and chart formats. A legend is also included to explain the meaning of each colour. In addition, users can interact directly with the chart legend labelled "Type" to select the data they wish to view without using the dropdown menu.

Overall, this dashboard makes the data more accessible to a wider audience. Unlike a static image, it allows users to explore the charts interactively by hovering over data points and adjusting the displayed variables. This interactivity helps users gain a deeper understanding of what Nightingale sought to demonstrate that preventable diseases caused far more deaths among soldiers than battle wounds.

In [14]:
# Run this in a Jupyter Notebook cell

import pandas as pd
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import webbrowser

# --- Load and prepare data ---
nightingdata = pd.read_csv("data/nightingale.csv")
nightingdata.sort_values(by='Date', inplace=True)

# Split into two ranges
data_1854_1855 = nightingdata[nightingdata['Date'] < '1855-04-01']
data_1855_1856 = nightingdata[nightingdata['Date'] >= '1855-04-01']

# Melt for plotting
def melt_data(df):
    melted = df.melt(
        id_vars=['Date', 'Month', 'Year', 'Army'],
        value_vars=['Disease', 'Wounds', 'Other'],
        var_name='Type',
        value_name='Deaths'
    )
    return melted

viz_1854_1855 = melt_data(data_1854_1855)
viz_1855_1856 = melt_data(data_1855_1856)

# --- Initialize Dash app ---
app = Dash(__name__)

# --- Layout ---
app.layout = html.Div(
    style={
        'textAlign': 'center',
        'fontFamily': 'Segoe UI, sans-serif',
        'backgroundColor': '#F8F9FB',
        'padding': '30px'
    },
    children=[
        html.H1(
            "Florence Nightingale's Data Dashboard",
            style={'color': '#2C3E50', 'marginBottom': '10px'}
        ),
        html.P(
            "Explore deaths during the Crimean War using Nightingale’s Coxcomb or Stacked Bar Graph",
            style={'color': '#555', 'fontSize': '18px', 'marginBottom': '30px'}
        ),

        # --- Controls section ---
        html.Div([
            html.Div([
                html.Label("Select chart type:", style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id='chart-type-dropdown',
                    options=[
                        {'label': 'Coxcomb (Polar)', 'value': 'polar'},
                        {'label': 'Stacked Bar', 'value': 'bar'}
                    ],
                    value='polar',
                    clearable=False,
                    style={'width': '100%'}
                ),
            ], style={'width': '45%', 'display': 'inline-block', 'marginRight': '20px'}),

            html.Div([
                html.Label("Select type of death:", style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id='type-dropdown',
                    options=[{'label': t, 'value': t} for t in ['All'] + viz_1854_1855['Type'].unique().tolist()],
                    value='All',
                    clearable=False,
                    style={'width': '100%'}
                ),
            ], style={'width': '45%', 'display': 'inline-block'}),
        ], style={'marginBottom': '25px'}),

        html.Button(
            'Reset Filters',
            id='reset-button',
            n_clicks=0,
            style={
                'backgroundColor': '#2C3E50',
                'color': 'white',
                'padding': '10px 24px',
                'border': 'none',
                'borderRadius': '8px',
                'cursor': 'pointer',
                'fontSize': '16px',
                'marginBottom': '30px'
            }
        ),

        # --- Charts and Legend section ---
        html.Div([
            # --- Charts container ---
            html.Div([
                html.Div([
                    html.H3("Apr 1854 to Mar 1855", style={'textAlign': 'center', 'color': '#2C3E50'}),
                    dcc.Graph(id='chart-1854-1855', style={'height': '700px'})
                ], style={
                    'display': 'inline-block',
                    'width': '48%',
                    'backgroundColor': 'white',
                    'padding': '15px',
                    'borderRadius': '10px',
                    'boxShadow': '0 2px 10px rgba(0,0,0,0.1)',
                    'margin': '0 1%',
                    'verticalAlign': 'top'
                }),

                html.Div([
                    html.H3("Apr 1855 to Mar 1856", style={'textAlign': 'center', 'color': '#2C3E50'}),
                    dcc.Graph(id='chart-1855-1856', style={'height': '700px'})
                ], style={
                    'display': 'inline-block',
                    'width': '48%',
                    'backgroundColor': 'white',
                    'padding': '15px',
                    'borderRadius': '10px',
                    'boxShadow': '0 2px 10px rgba(0,0,0,0.1)',
                    'margin': '0 1%',
                    'verticalAlign': 'top'
                })
            ], style={'display': 'inline-block', 'verticalAlign': 'top'}),

            # --- Legend / Explanation panel ---
            html.Div([
                html.H3("Legend", style={'color': '#2C3E50', 'textAlign': 'center'}),
                html.P("🟦 Blue – Deaths from preventable diseases", style={'color': '#333', 'fontSize': '15px'}),
                html.P("🟥 Red – Deaths from wounds", style={'color': '#333', 'fontSize': '15px'}),
                html.P("🟩 Green – Deaths from other causes", style={'color': '#333', 'fontSize': '15px'}),
                html.P("Data adapted from Florence Nightingale’s 1858 report.",
                       style={'fontStyle': 'italic', 'color': '#555', 'fontSize': '14px', 'marginTop': '10px'})
            ], style={
                'display': 'inline-block',
                'width': '20%',
                'verticalAlign': 'top',
                'backgroundColor': 'white',
                'padding': '20px',
                'marginLeft': '20px',
                'borderRadius': '10px',
                'boxShadow': '0 2px 10px rgba(0,0,0,0.1)',
                'textAlign': 'left'
            })
        ], style={'textAlign': 'center'})
    ]
)

# --- Callback to update charts ---
@app.callback(
    Output('chart-1854-1855', 'figure'),
    Output('chart-1855-1856', 'figure'),
    Input('chart-type-dropdown', 'value'),
    Input('type-dropdown', 'value')
)
def update_charts(chart_type, selected_type):
    def make_chart(df, title):
        if selected_type != 'All':
            df = df[df['Type'] == selected_type]
        
        color_map = {
            "Disease": "#4C78A8",   # Blue
            "Wounds": "#E45756",    # Red
            "Other": "#54A24B"      # Green
        }

        if chart_type == 'polar':
            fig = px.bar_polar(
                df,
                r='Deaths',
                theta='Month',
                color='Type',
                color_discrete_map=color_map,
                title=None
            )
            fig.update_traces(marker_line_color='white', marker_line_width=1.5, opacity=0.9)
            fig.update_layout(
                polar_angularaxis_rotation=90,
                polar=dict(
                    bgcolor='rgb(245,245,245)',
                    angularaxis=dict(showline=True, linecolor='lightgray'),
                    radialaxis=dict(showline=True, gridcolor='white')
                ),
                margin=dict(l=20, r=20, t=20, b=20),
                showlegend=True
            )
        else:
            fig = px.bar(
                df,
                x='Month',
                y='Deaths',
                color='Type',
                color_discrete_map=color_map,
                barmode='stack',
                title=None
            )
            fig.update_layout(
                margin=dict(l=20, r=20, t=20, b=20),
                showlegend=True
            )
        return fig

    fig1 = make_chart(viz_1854_1855, "Apr 1854 to Mar 1855")
    fig2 = make_chart(viz_1855_1856, "Apr 1855 to Mar 1856")
    return fig1, fig2

# --- Reset filters ---
@app.callback(
    Output('chart-type-dropdown', 'value'),
    Output('type-dropdown', 'value'),
    Input('reset-button', 'n_clicks'),
    prevent_initial_call=True
)
def reset_filters(n_clicks):
    return 'polar', 'All'

# --- Run the app ---
app.run(debug=False, use_reloader=False, port=8053)
webbrowser.open_new_tab("http://127.0.0.1:8053/")


True